## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-10-36-32 +0000,bbc,Conservative MP Danny Kruger defects to Reform UK,https://www.bbc.com/news/articles/ce802dmgnyro...
1,2025-09-15-10-33-50 +0000,bbc,When is the Budget and what might be in it?,https://www.bbc.com/news/articles/cj4w44w42j5o...
2,2025-09-15-10-27-07 +0000,wapo,"Rubio visits Israel, expected to visit Qatar a...",https://www.washingtonpost.com/national-securi...
3,2025-09-15-10-26-41 +0000,bbc,Conor McGregor ends bid to be Irish president,https://www.bbc.com/news/articles/cly9rnv54jno...
4,2025-09-15-10-26-35 +0000,nypost,Zohran Mamdani shares eerie similarities to fo...,https://nypost.com/2025/09/15/us-news/zohran-m...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
87,trump,28
231,kirk,21
314,charlie,15
232,suspect,8
82,will,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
174,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...,90
108,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,79
148,2025-09-14-19-31-04 +0000,nypost,Eric Trump: Assassination of Charlie Kirk has ...,https://nypost.com/2025/09/14/us-news/eric-tru...,77
191,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...,76
119,2025-09-14-22-55-24 +0000,nypost,"2 men arrested for allegedly planting, lightin...",https://nypost.com/2025/09/14/us-news/2-men-ar...,68


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
174,90,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...
197,35,2025-09-14-13-00-00 +0000,wsj,"As America’s 250th birthday approaches, Robert...",https://www.wsj.com/business/robert-reed-churc...
2,32,2025-09-15-10-27-07 +0000,wapo,"Rubio visits Israel, expected to visit Qatar a...",https://www.washingtonpost.com/national-securi...
78,32,2025-09-15-03-08-00 +0000,wsj,Signs of weakness in China’s economy stretched...,https://www.wsj.com/economy/chinas-economy-sho...
164,32,2025-09-14-17-33-45 +0000,nyt,White House Seeks More Supreme Court Security ...,https://www.nytimes.com/2025/09/14/us/politics...
71,30,2025-09-15-04-43-10 +0000,nypost,Maniac NYC driver who mowed down teen girl aft...,https://nypost.com/2025/09/15/us-news/maniac-n...
143,29,2025-09-14-19-55-03 +0000,nypost,Bank of New York claims Condé Nast space at On...,https://nypost.com/2025/09/14/business/bank-of...
65,26,2025-09-15-06-35-58 +0000,nypost,UK police say 49-year-old German remains a sus...,https://nypost.com/2025/09/15/world-news/uk-po...
120,26,2025-09-14-22-49-19 +0000,nypost,Roaming pack of wild dogs — as many as 40 — te...,https://nypost.com/2025/09/14/us-news/roaming-...
182,26,2025-09-14-15-00-00 +0000,wsj,"Tariffs are rolling into the Treasury, but the...",https://www.wsj.com/economy/capital-gains-tax-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
